In [25]:
import bm_preproc
from bm_preproc import BoyerMoore
import bisect

In [26]:
def readGenome(filename):
    """ Read in the genome from the input FASTA file """
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [27]:
def bm_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p. Count the number of
        alignments and character comparisons"""
    i = 0
    occurrences = []
    num_alignments = 0
    num_char_comps = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_char_comps += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
        num_alignments += 1
    return occurrences, num_alignments, num_char_comps

In [28]:
""" Question 2. How many character comparisons does the naive exact matching algorithm 
try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG to the excerpt 
of human chromosome 1?  """

""" Question 3. How many alignments does Boyer-Moore try when matching the string 
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG to the excerpt of human chromosome 1?  """

p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = readGenome('chr1.GRCh38.excerpt.fasta')
dna_alphabet = 'ACGT'
p_bm = BoyerMoore(p, dna_alphabet)
occurrences, num_alignments, num_character_comparisons = bm_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 127974 165191


In [29]:
def naive_with_counts(p, t):
    """ Naive exact matching algorithm. Count the number
        of alignments and character comparisons"""
    occurrences = []
    num_alignments = 0
    num_char_comps = 0
    for i in range(len(t) - len(p) + 1):  
        num_alignments += 1
        match = True
        for j in range(len(p)):
            num_char_comps += 1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)  
    return occurrences, num_alignments, num_char_comps

In [30]:
""" Question 1. How many alignments does the naive exact matching algorithm try when matching the string 
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG to the excerpt of human chromosome 1? """

p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = readGenome('chr1.GRCh38.excerpt.fasta')
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


In [31]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [41]:
""" Question 4. Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, 
    finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not 
    allowed. Don't consider any reverse complements. """

def approximate_match(p, t, n):
    segment_length = round(len(p)) // (n+1)
    all_matches = set()
    hits = 0
    idx = Index(t, 8)
    
    for i in range(n+1):
        start = i * segment_length
        end = min((i+1) * segment_length, len(p))
        matches = idx.query(p[start : end])
        hits += len(matches)

        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue

            mismatches = 0
            for j in range(0, start):
                if p[j] != t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if p[j] != t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break

            if mismatches <= n:
                all_matches.add(m - start)

    return list(all_matches), hits 

In [42]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = readGenome('chr1.GRCh38.excerpt.fasta')
all_matches, hits = approximate_match(p, t, 2)
print(len(all_matches))

19


In [43]:
""" Question 5. Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of 
    GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1? """

hits

90

In [44]:
""" Question 6. Write a function that, given a length-24 pattern P and given a SubseqIndex object 
    built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches. """

class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [56]:
def approximate_match_subseq(p, t, n):
    segment_length = round(len(p)) // (n+1)
    all_matches = set()
    hits = 0
    subseq_idx = SubseqIndex(t, 8, 3)    # same as other approx. match function, only diff is SubseqIndex in place of Index
    
    for i in range(n+1):
        matches = subseq_idx.query(p[i:])
        hits += len(matches)

        for m in matches:
            if m < i or m-i+len(p) > len(t):
                continue

            mismatches = 0
            for j in range(0, len(p)):
                if p[j] != t[m-i+j]:
                    mismatches += 1
                    if mismatches > n:
                        break

            if mismatches <= n:
                all_matches.add(m - i)

    return list(all_matches), hits 

In [58]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = readGenome('chr1.GRCh38.excerpt.fasta')
_, hits = approximate_match_subseq(p, t, 2)
hits

79